# Perceptron

In [2]:
import numpy as np
import iarray as ia

In [3]:
class Perceptron(object):

    def __init__(self, no_of_inputs, threshold=100, learning_rate=0.01):
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = ia.zeros(no_of_inputs)
        self.weights0 = 0
           
    def predict(self, inputs):
        summation = ia.matmul(inputs, self.weights) + self.weights0
        if summation > 0:
          activation = 1
        else:
          activation = 0            
        return activation

    def train(self, training_inputs, labels):
        for _ in range(self.threshold):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                self.weights += self.weights + self.learning_rate * (label - prediction) * inputs
                self.weights0 += self.weights0 + self.learning_rate * (label - prediction)

In [5]:
training_inputs = []
training_inputs.append(ia.numpy2iarray(np.array([1, 1])))
training_inputs.append(ia.numpy2iarray(np.array([1, 0])))
training_inputs.append(ia.numpy2iarray(np.array([0, 1])))
training_inputs.append(ia.numpy2iarray(np.array([0, 0])))

labels = np.array([1, 0, 0, 0])

training_inputs = ia.numpy2iarray(training_inputs)
labels = ia.numpy2iarray(labels)

perceptron = Perceptron(2)
perceptron.train(training_inputs, labels)

inputs = np.array([1, 1])
perceptron.predict(inputs) 
#=> 1

inputs = np.array([0, 1])
perceptron.predict(inputs) 

NotImplementedError: Only float32 and float64 types are supported for now